In [ ]:
import os
from keras.models import load_model,Model
from keras.layers import Lambda,Dense,Input
import keras.backend as K
import librosa.display
import cv2
import librosa
import matplotlib.pyplot as plt
import numpy as np
import string
import random
import tensorflow as tf

def create_spectrogram(clip,sample_rate,save_path):
  plt.interactive(False)
  fig=plt.figure(figsize=[0.72,0.72])
  ax=fig.add_subplot(111)
  ax.axes.get_xaxis().set_visible(False)
  ax.axes.get_yaxis().set_visible(False)
  ax.set_frame_on(False)
  S=librosa.feature.melspectrogram(y=clip,sr=sample_rate)
  librosa.display.specshow(librosa.power_to_db(S,ref=np.max))
  fig.savefig(save_path,dpi=400,bbox_inches='tight',pad_inches=0)
  plt.close()
  fig.clf()
  plt.close(fig)
  plt.close('all')
  del save_path,clip,sample_rate,fig,ax,S

def load_and_preprocess_image(path):
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Could not load image at {path}")
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0  # Normalize pixel values
    return img

In [ ]:
model=tf.keras.models.load_model('embdmodel.keras', safe_mode=False)
model.summary(expand_nested=True,
    show_trainable=True,)

Model: "functional_95"

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)      ┃ Output Shape    ┃   Param # ┃ Connected to   ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_22    │ (None, 150,     │         0 │ -              │   -   │
│ (InputLayer)      │ 150, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ input_layer_23    │ (None, 150,     │         0 │ -              │   -   │
│ (InputLayer)      │ 150, 3)         │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ sequential_7      │ (None, 64)      │    93,248 │ input_layer_2… │   Y   │
│ (Sequential)      │                 │           │ input_layer_2… │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ conv2d_28    │ (None, 148,     │       896 │ -              │   Y   │
│ (Conv2D)          │ 148, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ dropout_28   │ (None, 148,     │         0 │ -              │   -   │
│ (Dropout)         │ 148, 32)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ conv2d_29    │ (None, 146,     │    18,496 │ -              │   Y   │
│ (Conv2D)          │ 146, 64)        │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └              │ (None, 73, 73,  │         0 │ -              │   -   │
│ max_pooling2d_14  │ 64)             │           │                │       │
│ (MaxPooling2D)    │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ dropout_29   │ (None, 73, 73,  │         0 │ -              │   -   │
│ (Dropout)         │ 64)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ conv2d_30    │ (None, 71, 71,  │    36,928 │ -              │   Y   │
│ (Conv2D)          │ 64)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ dropout_30   │ (None, 71, 71,  │         0 │ -              │   -   │
│ (Dropout)         │ 64)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ conv2d_31    │ (None, 69, 69,  │    36,928 │ -              │   Y   │
│ (Conv2D)          │ 64)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └              │ (None, 34, 34,  │         0 │ -              │   -   │
│ max_pooling2d_15  │ 64)             │           │                │       │
│ (MaxPooling2D)    │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └ dropout_31   │ (None, 34, 34,  │         0 │ -              │   -   │
│ (Dropout)         │ 64)             │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│    └              │ (None, 64)      │         0 │ -              │   -   │
│ global_max_pooli… │                 │           │                │       │
│ (GlobalMaxPoolin… │                 │           │                │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ lambda_7 (Lambda) │ (None, 64)      │         0 │ sequential_7[… │   -   │
│                   │                 │           │ sequential_7[… │       │
├───────────────────┼─────────────────┼───────────┼────────────────┼───────┤
│ dense_7 (Dense)   │ (None, 1)       │        65 │ lambda_7[0][0] │   Y 

 Total params: 279,941 (1.07 MB)

 Trainable params: 93,313 (364.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 186,628 (729.02 KB)

In [ ]:
embedding_model=model.layers[2]
embedding_model.summary(expand_nested=True,
    show_trainable=True,)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ conv2d_28 (Conv2D)          │ (None, 148, 148, 32)  │        896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_28 (Dropout)        │ (None, 148, 148, 32)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv2d_29 (Conv2D)          │ (None, 146, 146, 64)  │     18,496 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ max_pooling2d_14            │ (None, 73, 73, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_29 (Dropout)        │ (None, 73, 73, 64)    │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv2d_30 (Conv2D)          │ (None, 71, 71, 64)    │     36,928 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_30 (Dropout)        │ (None, 71, 71, 64)    │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv2d_31 (Conv2D)          │ (None, 69, 69, 64)    │     36,928 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ max_pooling2d_15            │ (None, 34, 34, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_31 (Dropout)        │ (None, 34, 34, 64)    │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_max_pooling2d_7      │ (None, 64)            │          0 │   -   │
│ (GlobalMaxPooling2D)        │                       │            │       │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 93,248 (364.25 KB)

 Trainable params: 93,248 (364.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Read the songs,divide them into 10s segment,create spectrogram of them

charsets=string.ascii_letters

songspecdict={}

os.makedirs('./Test_Spectrograms/', exist_ok=True)
songs_path = '/content/Music/'
songs_list = [f for f in os.listdir(songs_path) if os.path.isfile(os.path.join(songs_path, f))] # Filter for files only

def get_random_name():
    name=''.join([random.choice(charsets) for _ in range(20)])
    name=name+str(np.random.randint(0,1000))
    return name

for song in songs_list:
    print(song)
    songfile,sr=librosa.load(songs_path+song)
    duration=librosa.get_duration(y=songfile,sr=sr)
    prev=0
    emblist=[]
    for i in range(1,int((duration//10)+1)):
        if i==int((duration//10)):
            """Since we are dividing the song in 10s segment there might be case that after taking 10
            fragments also few more seconds are left so in this case extra becomes extra=extra+(10-extra)
            from the previous segment."""
            extra=int((int(duration)/10-int(int(duration)/10))*10)
            st=(sr*i*10)-(10-extra)
            end=st+10
            songfrag=np.copy(songfile[st:end])
        else:
            songfrag=np.copy(songfile[prev:(sr*i*10)])
        specname=get_random_name()
        create_spectrogram(songfrag,sr,'./Test_Spectrograms/'+specname+'.png')
        img_t=load_and_preprocess_image('./Test_Spectrograms/'+specname+'.png') # Added .png extension
        img_t=np.expand_dims(img_t,axis=0)
        emb=embedding_model.predict(img_t)
        emblist.append(emb)



        prev=sr*i*10
    songspecdict[song]=emblist


import pickle

with open('dict.pickle', 'wb') as handle:
    pickle.dump(songspecdict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Donell Jones - This Luv [cQUYe18YmSw].mp3
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Maxwell - 

/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=10
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Slow Dancing In A Burning Room (Live in L.A.) [32GZ3suxRn4].mp3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pickle
with open('dict.pickle', 'rb') as handle:
    songspecdict = pickle.load(handle)

In [ ]:
songs_path = '/content/Music/'
song,sr=librosa.load(songs_path+'Alicia Keys - Un-thinkable (I\'m Ready) (Official Video).mp3')
to_match=np.copy(song[0:220500])
create_spectrogram(to_match,22050,'test.png')
songsdistdict={}

to_match_img=load_and_preprocess_image('test.png')
to_match_img=np.expand_dims(to_match_img,axis=0)

to_match_emb=embedding_model.predict(to_match_img)

for key,values in songspecdict.items():
    dist_array=[]
    for embd in values:
        dist_array.append(np.linalg.norm(to_match_emb-embd))

    songsdistdict[key]=min(dist_array)

song_titles=list(songsdistdict.keys())
distances=list(songsdistdict.values())

print(f'Recognized Song={song_titles[distances.index(min(distances))]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Recognized Song=Alicia Keys - Un-thinkable (I'm Ready) (Official Video).mp3
